<a href="https://colab.research.google.com/github/jusimon/AlternusVera/blob/master/Common/FakeDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
from google.colab import drive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
googleDrive = GoogleDrive(gauth)
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
Colnames = ['Json_Id', 'Label', 'Statement', 'Subject', 'SpeakerName',
           'JobTitle', 'StateInfo', 'partyAffiliation',
           'BarelyTrueCounts', 'FalseCounts','HalfTrueCounts',
           'MostlyTrueCounts','PantsOnFireCounts','Context']

valid_link = "https://drive.google.com/open?id=1gaS9R-PQvER5DE1dAgh7z6v4u_euIvrO"
fluff, id3 = valid_link.split('=')
valid_filename = 'valid.tsv'
downloaded = googleDrive.CreateFile({'id':id3}) 
downloaded.GetContentFile(valid_filename)
valid_news = pd.read_csv(valid_filename, sep='\t', names = Colnames,error_bad_lines=False)

In [0]:
Factors = [
           {
               "name" : "Political_Affiliation",
               "git" : "https://github.com/jusimon/AlternusVera",
               "folder" :"AlternusVera",
               "file" : "Political_Affiliation",
               "model" : "/content/gdrive/My Drive/Drifters/Models/Politcal_Affiliation_Model.pkl"
               }   
           ]

In [5]:
!rm -rf AlternusVera
!git clone https://github.com/jusimon/AlternusVera
!cp AlternusVera/Political_Affiliation.py ./

Cloning into 'AlternusVera'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 133 (delta 0), reused 0 (delta 0), pack-reused 129
Receiving objects: 100% (133/133), 34.16 MiB | 19.20 MiB/s, done.
Resolving deltas: 100% (60/60), done.


In [7]:
!rm -rf AlternusVera
!git clone https://github.com/psaini79/AlternusVera
!cp AlternusVera/sensational.py ./

Cloning into 'AlternusVera'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 139 (delta 6), reused 10 (delta 4), pack-reused 127
Receiving objects: 100% (139/139), 98.27 MiB | 35.29 MiB/s, done.
Resolving deltas: 100% (62/62), done.
Checking out files: 100% (11/11), done.


In [6]:
!rm -rf AlternusVera
!git clone https://github.com/viswa0531/AlternusVera
!cp AlternusVera/Biased_Factor.py ./

Cloning into 'AlternusVera'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 140 (delta 0), reused 0 (delta 0), pack-reused 137
Receiving objects: 100% (140/140), 9.63 MiB | 22.31 MiB/s, done.
Resolving deltas: 100% (68/68), done.


In [8]:
from Political_Affiliation import Political_Affiliation
from sensational import sensational
from Biased_Factor import BiasedDetection

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


##### This function calculates fakeness percentage using the weighted_percentage function passing all factors fakeness score. It calls respective factors class to get the fakeness score of individual factor

In [0]:
from gensim.models.doc2vec import Doc2Vec
def fakecheck(dataFrame):
  biasedFactor = BiasedDetection(dataFrame).predict()
  PoliticalAff = Political_Affiliation(valid_news).predict()
  #  Authenticity = Authenticity(dataFrame).predict()
  # Sensationalism = sensational(valid_news).predict()
  print('biased Factor:',biasedFactor)    
  print('political_affilication_factor:',PoliticalAff)
  #  print('spelling_error_factor:',spelling_error_factor)
  #  print('spam_detection_factor:',spam_detection_factor)
  #  print('stance_detection_factor:',stance_detection_factor)
  return weighted_percentage(0, PoliticalAff, biasedFactor, 0)

In [0]:
def weighted_percentage(SensationalismF, politicalAffilitaionF, BiasedF, AuthenticityF):                       
    SensationalismF_Weigth=.84
    BiasedF_Weight=.53
    politicalAffilitaionF_Weight=.65
    AuthenticityF_Weight=.75    
    total_weight=SensationalismF_Weigth+BiasedF_Weight+politicalAffilitaionF_Weight+AuthenticityF_Weight
    print(total_weight)
    return round((((SensationalismF*SensationalismF_Weigth)+
            (politicalAffilitaionF*politicalAffilitaionF_Weight)+
            (BiasedF*BiasedF_Weight)+
            (AuthenticityF*AuthenticityF_Weight))/total_weight)*100,2)

In [0]:
print(weighted_percentage(1,1,1,1))
print(weighted_percentage(0,0,0,0))
print(weighted_percentage(.8,.6,1,.7))

100.0
0.0
76.43


#### Check Fakeness of a news article

In [14]:
#input_news='Obamacare will provide insurance to all non-U.S. residents, even if they are here illegally.A strong bipartisan majority in the House of Representatives voted to defund Obamacare.07732584351 - Rodger Burns - MSG = We tried to call you re your reply to our sms for a free nokia mobile + free camcorder. Please call now 08000930705 for delivery tomorrow.Amazon Enters eBook Subscription Market with Kindle Unlimited'
input_news = 'former senator'
fakecheck(input_news)

biased Factor: 1
political_affilication_factor: 0.5218068535825545


31.38